In [1]:
import csv
import numpy as np
import pandas as pd
import random
import json
import requests
from sklearn.preprocessing import StandardScaler

In [19]:
## set credentials and endpoint
username = "default\\admin"
password = "Test123!"
url = "https://vsystem.ingress.dh-rzsmhj7iwkz.di-dev2.shoot.canary.k8s-hana.ondemand.com/app/pipeline-modeler/openapi/service/f55cfd17-573a-4161-b21d-3f47ec88e93e/"
path_predict = "v1/predict/"
path_predict_classes = "v1/predict_classes/"

def predict(test, predict_classes=False):
    global username, password, url, numberClassifications
    # call the REST api to predict
    headers = {
                'Content-Type': 'application/json',
                'X-Requested-With': 'XMLHttpRequest'
              }
    payload = json.loads('{"data": ' + str(test[0:random.randrange(1,10,2)].tolist()) + '}')
    print(payload)
    url2 = ""
    if predict_classes:
        url2 = url + path_predict_classes
    else:
        url2 = url + path_predict
    resp = requests.post(url2, auth=(username, password), headers=headers, json=payload)

    return resp

In [23]:
test_df = pd.read_csv('titanic_test.csv')
X_test = test_df.drop("alone", axis = 1).to_numpy()
str(X_test[0:random.randrange(1,10,2)].tolist())

'[[3, 0, 2, 0, 0, 1, 1, 1], [2, 0, 0, 1, 1, 0, 0, 0], [3, 1, 0, 3, 2, 0, 0, 0], [3, 1, 2, 0, 1, 0, 2, 0], [3, 1, 4, 0, 1, 0, 2, 0]]'

In [21]:
payload = '{"data": ' + str(X_test[0:3].tolist()) + '}'
feed = json.loads(payload)
input_data = np.array(feed['data'])
scaler = StandardScaler()
scaler.fit(X_test)
X_test = scaler.transform(input_data)
X_test

array([[ 0.88673266, -0.73343782, -0.36543665, -0.51463306, -1.24754424,
         1.01703274, -0.36951512,  0.7922987 ],
       [-0.31900748, -0.73343782, -1.26590485,  0.39618577, -0.44811215,
        -0.53638028, -2.08622079, -1.26215025],
       [ 0.88673266,  1.3634421 , -1.26590485,  2.21782344,  0.35131993,
        -0.53638028, -2.08622079, -1.26215025]])

In [36]:
response = predict(X_test, True)
print(response.text)
for prediction in json.loads(response.text)['Results']:
    print(prediction)

{'data': [[3, 0, 2, 0, 0, 1, 1, 1], [2, 0, 0, 1, 1, 0, 0, 0], [3, 1, 0, 3, 2, 0, 0, 0], [3, 1, 2, 0, 1, 0, 2, 0], [3, 1, 4, 0, 1, 0, 2, 0], [1, 1, 6, 1, 3, 1, 2, 0], [3, 0, 6, 0, 1, 0, 1, 1], [2, 0, 6, 0, 1, 0, 1, 1], [3, 1, 4, 1, 2, 2, 2, 0]]}
{"Results": ["0", "1", "0", "1", "0", "1", "0", "0", "1"]}
0
1
0
1
0
1
0
0
1
